In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
package_folder='C:/Users/admin/Documents/Nitin/others/unittest/src'
# in_folder='/Users/rossmoore/Documents/code/prophet/ChrisLai'
# in_folder='/Users/romoore/Resilio Sync/code/prophet/ChrisLai'
# in_folder='/media/Media/Documents/code/Resilio_code/prophet/dais/Examples/Traditional'
in_folder='C:/Users/admin/Documents/Nitin/others/unittest/test_data_folder'
tables_file='TablesCommon.xlsx'
tables_be_file='TablesBE.xlsx'
maps_file='Table_Maps.xlsx'
mpf_file='MPF.txt'
# mpf_file10k='MPF10k.parquet'
# mpf_file50k='MPF50k.parquet'

In [3]:
sys.path.insert(0, package_folder)
import dais

In [4]:
from dais.models.traditional2 import TraditionalLifeFunctions
from dais.models.common3 import BaseModel, BondModel
# ,FunctionNode, ModelDependencyGraph

C:\Users\admin\Anaconda3\lib\site-packages\numba\decorators.py:149: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [5]:
a=BaseModel()
# a=BondModel()

In [6]:
%%time
a.setfuncs(TraditionalLifeFunctions)

162 variable definitions imported.
Wall time: 5.02 ms


In [7]:
mpf=pd.read_table(os.path.join(in_folder,mpf_file),header=0)
# mpf=pd.read_parquet(os.path.join(in_folder,mpf_file10k))
def cp_type(x):
    if x[:1]=="A":
        return 0
    elif x[:2]=="L-":
        return 1
    elif x[:2]=="Le":
        return 2
    else:
        return 999999
def cp_type_adj(x):
    if x[:1]=="A":
        return 0
    elif x[:2]=="L-":
        return 1
    elif x[:2]=="Le":
        return 2
    elif x[:2]=="J-":
        return 3
    else:
        return 999999
mpf=mpf.assign(BASE_PROD_CODE_patch=mpf['BASE_PROD_CODE'].str.split('_').str.get(-1).astype(np.int64)
            ,CP_TYPE_ADJ_patch=mpf['CP_TYPE_ADJ'].apply(cp_type_adj)
            , CP_TYPE_patch=mpf['CP_TYPE'].apply(cp_type)
            , ISS_YR_IDX=999999)
a.add_source_mpf(mpf)

Processing MPF. 112 unmapped variables to start with.


AttributeError: 'Series' object has no attribute 'to_numpy'

In [20]:
a.add_source_tables(pd.read_excel(os.path.join(in_folder,tables_be_file),sheet_name=None))
a.add_source_tables(pd.read_excel(os.path.join(in_folder,tables_file),sheet_name=None))

Processing tables. 61 unmapped variables to start with.
Processed tables. 26 new variables mapped.
Processing tables. 35 unmapped variables to start with.
Processed tables. 14 new variables mapped.


In [21]:
CONSTANTS={
    'max_incidence_lkup_age': 126,
    'max_mort_sel_lkup_yr': 25,
    'max_lapse_lkup_yr': 21,
    'min_ppia_pc_lkup_yr':1998,
    'max_ppia_pc_lkup_yr':2017,
    'val_yr':2017,
    'val_mth':12,
    'BE_INDEX':2,
    'COMM_LIMIT_ADJ_index':1,
    'IC_PERIOD_M':12,
    'MIN_TAX_LKUP_Y':1996,
    'MAX_TAX_LKUP_Y':2020,
    'MAX_COMM_MILE_M':120,
    'COMM_MILE_FAC':0.25,
    'RC_START_M':13,
    'MIN_COMM_ID':5,
    'M_DISC_RATE':(1+0.06)**(1/12)-1,
    't_range':1200,
    'SPIKE_RATE_PERIOD_INDEX':1,
    'MORT_PAD_PC':0,
    'DEC_WOP_PAD_PC':0,
    'MAX_EXP_LKUP_Y':20,
    'ASSPT_BE_index':1,
    'COM_YEAR_END':3,
    'COMM_CB_ST_YM':201510,
    'COMM_CB_ADJ_RATE':0.4,
    'MAX_COMM_CB_M':48
}
a.add_source_constants(CONSTANTS)

Processing constants. 21 unmapped variables to start with.
Processed tables. 21 new variables mapped.


In [ ]:
#This line only works in Linux and Mac currently. 
# SVG(a.visualise(target_node='COMM_MILAGE_RATE',format='png'))

In [22]:
%%time
#This line will take *minutes* the first time it is run on a machine. Should be very fast after that.
a.initialise()


Caching:  LAPSE_PAD_PC
Initialising:  LAPSE_PAD_PC
Caching:  SPIKE_LAPSE_PERIOD2
Initialising:  SPIKE_LAPSE_PERIOD2
Caching:  LAPSE_CHAN_IDX
Initialising:  LAPSE_CHAN_IDX
Caching:  COMM_CB_CODE_patch
Initialising:  COMM_CB_CODE_patch
Caching:  REN_COMM_S_AG_FAC
Initialising:  REN_COMM_S_AG_FAC
Caching:  COMM_S_AG_FAC
Initialising:  COMM_S_AG_FAC
Caching:  REN_COMM_AG_FAC_1_derived
Initialising:  REN_COMM_AG_FAC_1_derived
Caching:  COMM_AG_FAC_1_derived
Initialising:  COMM_AG_FAC_1_derived
Caching:  MORT_SMK_IDX
Initialising:  MORT_SMK_IDX
Caching:  WOP_EXP_SEX_IDX
Initialising:  WOP_EXP_SEX_IDX
Caching:  WOP_SEX_IDX
Initialising:  WOP_SEX_IDX
Caching:  MORT_RATE_MAX_AGE
Initialising:  MORT_RATE_MAX_AGE
Caching:  MORT_SEX_IDX
Initialising:  MORT_SEX_IDX
Caching:  MORT_SEL_SEX_IDX
Initialising:  MORT_SEL_SEX_IDX
Caching:  PREM_FREQ
Initialising:  PREM_FREQ
Caching:  COMM_MILE_PC_derived
Initialising:  COMM_MILE_PC_derived
Caching:  IC_ADD_SPEC_PC_derived
Initialising:  IC_ADD_SPEC_PC_der

In [23]:
%%time
#This will take 10-20 seconds regardless of number of policies the first time it is run on a machine
#Subsequent runs should be at full speed.
a.execute(dokeep=set(['sum_DISC_PREM_INC','DISC_PREM_INC','DISC_TOT_COMM']))

Wall time: 52.2 s


In [24]:
#This code helps the developer understand which functions take how much time
{k: v for k, v in sorted(a.timings.items(), key=lambda item: item[1])}

{'DEC_WOP_EXP_CODE': 0.001995086669921875,
 'DEC_WOP_IND': 0.002961874008178711,
 'DEC_WOP_SEX_IND': 0.0029861927032470703,
 'MORT_SEX_IND': 0.002991914749145508,
 'MORT_SEL_SEX_IND': 0.002991914749145508,
 'PPIA_DISC_PC': 0.0029921531677246094,
 'MORT_SEL_CODE': 0.002992391586303711,
 'LAPSE_CHAN_IND': 0.0030241012573242188,
 'JUV_IND': 0.003959178924560547,
 'DEC_WOP_CODE': 0.003988742828369141,
 'SPIKE_RATE_IND': 0.003989458084106445,
 'COMM_ADJ_MULT_TYPE': 0.003989696502685547,
 'MORT_CODE': 0.00399017333984375,
 'MORT_SMK_IND': 0.0039958953857421875,
 'LAPSE_CODE': 0.003996372222900391,
 'REN_RATE': 0.004019737243652344,
 'ANN_RATE': 0.004981279373168945,
 'LAPSE_PAD_CODE': 0.004983663558959961,
 'IC_ADD_BASE_IND': 0.004986286163330078,
 'IC_ADD_BASE_PC': 0.00498652458190918,
 'IC_ADD_SPEC_IND': 0.004987239837646484,
 'MAX_COMM_PAYBL_Y': 0.004989147186279297,
 'SPIKE_LAPSE_PERIOD': 0.0049896240234375,
 'COMM_LIMIT_ADJ': 0.005021095275878906,
 'LAPSE_PREM_FREQ_IND': 0.0059838294982

In [25]:
#You can access the calculated values as numpy arrays in this manner
#You will need to ensure the variable you want is "kept" in the dokeep option in the execute function (see above)
a.vals['DISC_PREM_INC'][19,:90]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.81298727, 0.80754902, 0.80213464,
       0.79674399, 0.7913769 , 0.78603324, 0.78071285, 0.77541557,
       0.77013726, 0.7648778 , 0.75963706, 0.7544149 , 0.74921121,
       0.74402585, 0.73885871, 0.73370966, 0.72857856, 0.72346531,
       0.71836976, 0.71329181, 0.70823135, 0.70318825, 0.69816239,
       0.69315365, 0.68816191, 0.68318704, 0.67822894, 0.67328747,
       0.66836252, 0.66345396, 0.65856169, 0.65368558, 0.64882